In [1]:
import json
import numpy as np

In [2]:
model = str(input("Enter the model file in .json format: "))
model = json.load(open(f"models/{model}", "r"))

layers = model["case"]["model"]
weights = model["case"]["weights"]
inputArray = model["case"]["input"]
inputArray = np.insert(inputArray, 0, 1)

# print("Model: ", layers)
# print("Weights: ", weights)
# print("Input: ", inputArray)

for i in range(len(layers["layers"])):
    layer = layers["layers"][i]
    numberOfNeurons = layer["number_of_neurons"]
    activationFunction = layer["activation_function"]

    if i == 0:
        weight = weights[0]
        input_data = inputArray
    else:
        weight = weights[i]
        input_data = sum
        # add bias
        input_data = np.insert(input_data, 0, 1)

    print("Layer: ", i)
    print("Number of Neurons: ", numberOfNeurons)
    print("Activation Function: ", activationFunction)
    print("Weight: ", weight)

    sum = np.zeros(numberOfNeurons)
    for j in range(numberOfNeurons):
        for k in range(len(input_data)):
            sum[j] += input_data[k] * weight[k][j]

        if activationFunction == 'relu':
            sum[j] = max(0, sum[j])
        elif activationFunction == 'sigmoid':
            sum[j] = 1 / (1 + np.exp(-sum[j]))
        elif activationFunction == 'linear':
            sum[j] = sum[j]
        elif activationFunction == 'softmax':
            sum[j] = np.exp(sum[j]) / np.sum(np.exp(sum[j]))
        
        print("Sum: h", j, " = ", sum[j])

    print(" ")



Layer:  0
Number of Neurons:  4
Activation Function:  relu
Weight:  [[0.1, 0.2, 0.3, -1.2], [-0.5, 0.6, 0.7, 0.5], [0.9, 1.0, -1.1, -1.0], [1.3, 1.4, 1.5, 0.1]]
Sum: h 0  =  2.09
Sum: h 1  =  1.22
Sum: h 2  =  0.2500000000000002
Sum: h 3  =  0.0
 
Layer:  1
Number of Neurons:  3
Activation Function:  relu
Weight:  [[0.1, 0.1, 0.3], [-0.4, 0.5, 0.6], [0.7, 0.4, -0.9], [0.2, 0.3, 0.4], [-0.1, 0.2, 0.1]]
Sum: h 0  =  0.16800000000000004
Sum: h 1  =  1.7080000000000002
Sum: h 2  =  0.5559999999999998
 
Layer:  2
Number of Neurons:  2
Activation Function:  relu
Weight:  [[0.1, 0.2], [-0.3, 0.4], [0.6, 0.1], [0.1, -0.4]]
Sum: h 0  =  1.1300000000000003
Sum: h 1  =  0.21560000000000012
 
Layer:  3
Number of Neurons:  1
Activation Function:  sigmoid
Weight:  [[0.1], [-0.2], [0.3]]
Sum: h 0  =  0.4846748017763877
 
